In [1]:
import time
import random
import tqdm


import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance
from geopy.extra.rate_limiter import RateLimiter

pd.set_option('max_colwidth', 600)
pd.set_option('display.width', 600)

In [2]:
df = pd.read_csv('df_2.csv', index_col=0)

In [3]:
df['date'] = df['text'].apply(lambda x: x.split('.  Продам')[0])
df['note_len'] = df['text'].apply(lambda x: len(x))
df = df.loc[df['note_len'] > 3]

df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y")

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofweek'] = df['date'].dt.dayofweek #Monday=0, Sunday=6

In [4]:
df['rooms'] = df['text'].apply(lambda x: x.split("Продам ")[1].split('квартиру')[0])

In [5]:
def district_sort(text):
    text = text.lower()
    if 'орджоникидз' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Орджоникидзевский(левый берег)'
        else:
            return 'Орджоникидзевский'
    elif 'правобереж' in text:
        return 'Правобережный'
    elif 'ленинск' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Ленинский(левый берег)'
        else:
            return 'Ленинский'

In [6]:
df['district'] = df['text'].apply(district_sort)
df['text'] = df['text'].map(lambda x: x.replace('ул. ул. ', 'ул. '))
df['street'] = df['text'].apply(lambda x: x.split("ул.")[1].split(', этаж')[0] if "ул." in x else 'unknown')
df['street'] = df['street'].map(lambda x: x.replace('ул ', ''))

df['street'] = df['street'].apply(lambda x: x.lstrip(". ").rstrip(", ")).map(lambda x: x.replace('дом ', ''))

In [7]:
df[['floor', 'total_floors']] = df['text'].apply(
    lambda x: x.split("этаж")[1].split(',')[0].split('.')[0] if "этаж" in x else 'unknown').str.split('/', expand=True)

In [8]:
df['total_area'] = df['text'].apply(lambda x: x.split("Общая площадь - ")[1].split(' кв.м.')[0] if "Общая площадь" in x else 'unknown')
df['living_area'] = df['text'].apply(lambda x: x.split("жилая площадь - ")[1].split(' кв.м.')[0] if "жилая площадь - " in x else 'unknown')
df['kitchen_area'] = df['text'].apply(lambda x: x.split("кухня - ")[1].split(' кв.м.')[0] if "кухня - " in x else 'unknown')

df['price'] = df['text'].apply(lambda x: x.split("Цена - ")[1].split(' т.р.')[0] if "Цена - " in x else None)

In [9]:
df['street'] = 'Магнитогорск, ' + df['street']

In [10]:
from tqdm import tqdm
tqdm.pandas()

geolocator = Nominatim(user_agent="my-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

df['location'] = df['street'].progress_apply(geocode)

 99%|█████████▊| 461/467 [15:22<00:11,  1.99s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Магнитогорск, Западное шоссе, 95',), **{}).
Traceback (most recent call last):
  File "e:\GITlocal\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "e:\GITlocal\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\s7\AppData\Local\Prog

In [11]:
df['lat_long'] = df['location'].dropna().apply(lambda x: x[1])

In [12]:
df.loc[df['district'].isna(), 'district'] = df['location'].dropna().apply(lambda x: x[0].split('район')[0].split(', ')[-1] if 'район' in str(x) else 'unknown')

In [35]:
parks_and_center = {'park_eko':(53.402233, 58.952641),
'attraction':(53.381266, 58.953624),
'south':(53.351887, 58.990688),
'eternal_fire':(53.405914, 58.991158),
'veter':(53.429143, 59.000213),
'city_center':(53.407164, 58.980285)}


In [37]:
# Итерирование по ключам словаря parks
for key in parks_and_center:
    # Создание нового столбца с именем ключа и заполнение его значениями
    df[key] = [(parks_and_center[key])]* len(df['lat_long'])
    df[key] = df[[key, 'lat_long']].dropna().apply(lambda row: round(distance.distance(row[key], row["lat_long"]).km, 2), axis=1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 462 entries, 0 to 467
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            462 non-null    datetime64[ns]
 1   day             462 non-null    int32         
 2   month           462 non-null    int32         
 3   year            462 non-null    int32         
 4   dayofweek       462 non-null    int32         
 5   rooms           462 non-null    int64         
 6   district        392 non-null    object        
 7   street          462 non-null    object        
 8   floor           462 non-null    float64       
 9   total_floors    462 non-null    float64       
 10  total_area      462 non-null    float64       
 11  living_area     462 non-null    float64       
 12  kitchen_area    462 non-null    float64       
 13  price           462 non-null    float64       
 14  lat_long        323 non-null    object        
 15  park_eko   

In [40]:
df['rooms'].value_counts()

rooms
 2    168
 3    144
 1    119
 4     19
 5      7
-1      5
Name: count, dtype: int64

In [41]:
df['rooms'][0]

2

In [42]:
def rooms_num(rooms):
    rooms = rooms.rstrip()
    answ = -1
    if rooms == 'однокомнатную':
        answ = 1
    if rooms == 'двухкомнатную':
        answ = 2
    if rooms == 'трехкомнатную':
        answ = 3
    if rooms == 'четырехкомнатную':
        answ = 4
    if rooms == 'многокомнатную':
        answ = 5
    return answ

In [43]:
df['rooms'] = df['rooms'].apply(rooms_num)

AttributeError: 'int' object has no attribute 'rstrip'

In [44]:
df['rooms']

0      2
1      2
2      1
3      5
4      2
      ..
463    3
464    2
465    2
466    2
467    3
Name: rooms, Length: 462, dtype: int64

In [45]:
df['district'].value_counts()

district
Орджоникидзевский                 152
Ленинский                         120
Правобережный                     103
Орджоникидзевский(левый берег)     14
unknown                             1
Ленинский(левый берег)              1
141-ый микро                        1
Name: count, dtype: int64

In [46]:
df.sample(2)

,date,day,month,year,dayofweek,rooms,district,street,floor,total_floors,...,kitchen_area,price,lat_long,park_eko,attraction,south,eternal_fire,veter,price_sq_meter,city_center
164,2023-07-24,24,7,2023,0,3,Орджоникидзевский,"Магнитогорск, Карла Маркса, 233",6.0,10.0,...,8.0,4850.0,"(53.3735251, 58.9769375)",3.58,1.77,2.58,3.73,6.38,67.832168,3.75
194,2023-07-12,12,7,2023,2,1,Орджоникидзевский,"Магнитогорск, Труда, 16",7.0,9.0,...,5.0,1400.0,"(53.3661724, 58.9917653)",4.78,3.04,1.59,4.42,7.03,65.727700,4.63


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 462 entries, 0 to 467
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            462 non-null    datetime64[ns]
 1   day             462 non-null    int32         
 2   month           462 non-null    int32         
 3   year            462 non-null    int32         
 4   dayofweek       462 non-null    int32         
 5   rooms           462 non-null    int64         
 6   district        392 non-null    object        
 7   street          462 non-null    object        
 8   floor           462 non-null    float64       
 9   total_floors    462 non-null    float64       
 10  total_area      462 non-null    float64       
 11  living_area     462 non-null    float64       
 12  kitchen_area    462 non-null    float64       
 13  price           462 non-null    float64       
 14  lat_long        323 non-null    object        
 15  park_eko   

In [48]:
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']] = \
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']].apply(pd.to_numeric, errors='coerce').dropna()

In [49]:
df['district'] = df['district'].dropna().apply(lambda x: x.lstrip().rstrip())

In [50]:
df = df.drop(['text',  'note_len', 'location'], axis=1)

KeyError: "['text', 'note_len', 'location'] not found in axis"

In [51]:
df['price'] = df['price'].dropna()

In [52]:
df = df.dropna(subset=['price'])

In [53]:
df.select_dtypes('number').corr()

,day,month,year,dayofweek,rooms,floor,total_floors,total_area,living_area,kitchen_area,price,park_eko,attraction,south,eternal_fire,veter,price_sq_meter,city_center
day,1.000000,0.160836,-0.100188,-0.064534,0.026026,-0.041467,-0.045312,-0.024267,0.044885,0.088851,0.044791,0.142866,0.092536,0.033466,0.132158,0.057308,0.016291,0.140893
month,0.160836,1.000000,-0.350154,-0.196757,0.065790,0.103072,0.102914,-0.024801,-0.051481,-0.080977,0.122701,-0.001378,0.044270,0.046140,-0.023065,-0.060362,0.161324,-0.027952
year,-0.100188,-0.350154,1.000000,-0.082181,-0.053646,0.069017,0.065414,-0.022944,-0.005992,-0.117143,-0.030961,-0.039081,-0.154337,-0.186024,0.007992,0.125073,-0.018055,0.012771
dayofweek,-0.064534,-0.196757,-0.082181,1.000000,-0.000594,-0.058815,-0.078184,0.031300,0.071268,-0.007854,-0.055679,0.064473,0.114805,0.083144,0.034451,-0.043120,-0.067981,0.038840
rooms,0.026026,0.065790,-0.053646,-0.000594,1.000000,-0.060396,-0.055408,0.561117,0.618774,0.153666,0.566696,0.134501,0.048384,-0.031478,0.122633,0.085731,-0.166420,0.140598
floor,-0.041467,0.103072,0.069017,-0.058815,-0.060396,1.000000,0.625597,0.073968,-0.018848,-0.007634,0.108879,-0.029814,-0.222779,-0.317661,0.028347,0.191312,0.112129,0.034870
total_floors,-0.045312,0.102914,0.065414,-0.078184,-0.055408,0.625597,1.000000,0.095331,0.015291,0.087349,0.261480,-0.042566,-0.367674,-0.506049,0.071217,0.313548,0.214163,0.077592
total_area,-0.024267,-0.024801,-0.022944,0.031300,0.561117,0.073968,0.095331,1.000000,0.538260,0.354774,0.573605,0.065071,-0.005291,-0.063794,0.074448,0.055635,-0.249429,0.082212
living_area,0.044885,-0.051481,-0.005992,0.071268,0.618774,-0.018848,0.015291,0.538260,1.000000,0.272692,0.509387,0.093155,0.037925,-0.036825,0.084645,0.060135,-0.117128,0.098315
kitchen_area,0.088851,-0.080977,-0.117143,-0.007854,0.153666,-0.007634,0.087349,0.354774,0.272692,1.000000,0.401155,0.062426,-0.106973,-0.179200,0.240135,0.261909,-0.001212,0.211907


In [54]:
df['price_sq_meter'] = df['price'] / df['total_area']

In [63]:
df['floor_type'] = 'middle'
df.loc[df.floor == 1, 'floor_type'] = 'first'
df.loc[(df.floor == df.total_floors) & (df.floor != 1), 'floor_type'] = 'last'

In [64]:
df.to_csv('df_3.csv') #ready for analysis